# 🔍 Fraud Detection - Exploratory Data Analysis

Анализ данных для системы детекции мошенничества

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from utils.data_loader import load_and_prepare_data
from features.feature_engineering import FraudFeatureEngineer

# Настройка отображения
pd.set_option('display.max_columns', 100)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

%matplotlib inline

## 1. Загрузка данных

In [ ]:
# Пути к данным
transactions_path = '../data/транзакции в Мобильном интернет Банкинге.csv'
behavioral_path = '../data/поведенческие паттерны клиентов.csv'

# Загрузка
df = load_and_prepare_data(transactions_path, behavioral_path)

## 2. Первичный анализ

In [ ]:
# Размер и структура
print(f"Размер датасета: {df.shape}")
print(f"\nПервые строки:")
df.head()

In [ ]:
# Информация о типах данных
df.info()

In [ ]:
# Статистика
df.describe()

## 3. Анализ таргета

In [ ]:
# Распределение таргета
fraud_counts = df['is_fraud'].value_counts()
fraud_rate = df['is_fraud'].mean()

print(f"Распределение is_fraud:")
print(fraud_counts)
print(f"\nДоля мошеннических транзакций: {fraud_rate:.4%}")

# Визуализация
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Bar plot
fraud_counts.plot(kind='bar', ax=ax[0], color=['green', 'red'])
ax[0].set_title('Распределение транзакций', fontsize=14)
ax[0].set_xlabel('Класс (0=чистая, 1=мошенничество)')
ax[0].set_ylabel('Количество')
ax[0].set_xticklabels(['Чистая', 'Мошенничество'], rotation=0)

# Pie chart
ax[1].pie(fraud_counts, labels=['Чистая', 'Мошенничество'], autopct='%1.2f%%', 
          colors=['green', 'red'], startangle=90)
ax[1].set_title('Доля мошеннических транзакций', fontsize=14)

plt.tight_layout()
plt.show()

## 4. Анализ суммы транзакций

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# Распределение суммы для чистых и мошеннических транзакций
clean_amounts = df[df['is_fraud'] == 0]['amount']
fraud_amounts = df[df['is_fraud'] == 1]['amount']

# Гистограммы
ax[0, 0].hist([clean_amounts, fraud_amounts], bins=50, label=['Чистая', 'Мошенничество'], 
              color=['green', 'red'], alpha=0.7)
ax[0, 0].set_xlabel('Сумма')
ax[0, 0].set_ylabel('Частота')
ax[0, 0].set_title('Распределение сумм')
ax[0, 0].legend()

# Log scale
ax[0, 1].hist([clean_amounts, fraud_amounts], bins=50, label=['Чистая', 'Мошенничество'], 
              color=['green', 'red'], alpha=0.7, log=True)
ax[0, 1].set_xlabel('Сумма')
ax[0, 1].set_ylabel('Частота (log scale)')
ax[0, 1].set_title('Распределение сумм (log scale)')
ax[0, 1].legend()

# Boxplot
df.boxplot(column='amount', by='is_fraud', ax=ax[1, 0])
ax[1, 0].set_xlabel('Класс')
ax[1, 0].set_ylabel('Сумма')
ax[1, 0].set_title('Boxplot сумм по классам')
ax[1, 0].set_xticklabels(['Чистая', 'Мошенничество'])

# Статистика
stats_text = f"""Чистые транзакции:
Средняя: {clean_amounts.mean():.2f}
Медиана: {clean_amounts.median():.2f}
Std: {clean_amounts.std():.2f}

Мошеннические:
Средняя: {fraud_amounts.mean():.2f}
Медиана: {fraud_amounts.median():.2f}
Std: {fraud_amounts.std():.2f}"""

ax[1, 1].text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center')
ax[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 5. Feature Engineering

In [ ]:
# Создание признаков
fe = FraudFeatureEngineer()
df_fe = fe.fit_transform(df.copy())

print(f"Размер после FE: {df_fe.shape}")
print(f"Новые признаки: {fe.feature_names}")

## 6. Корреляционный анализ

In [ ]:
# Корреляция численных признаков с таргетом
numeric_cols = df_fe.select_dtypes(include=[np.number]).columns
correlations = df_fe[numeric_cols].corr()['is_fraud'].sort_values(ascending=False)

print("Топ-20 признаков по корреляции с таргетом:")
print(correlations.head(20))

# Визуализация
plt.figure(figsize=(10, 8))
correlations.head(20).plot(kind='barh')
plt.xlabel('Корреляция с is_fraud')
plt.title('Топ-20 признаков по корреляции с таргетом')
plt.tight_layout()
plt.show()

## 7. Временные паттерны

In [ ]:
if 'hour' in df_fe.columns and 'day_of_week' in df_fe.columns:
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    
    # Распределение по часам
    hour_fraud = df_fe.groupby('hour')['is_fraud'].mean()
    hour_fraud.plot(kind='bar', ax=ax[0], color='red')
    ax[0].set_xlabel('Час')
    ax[0].set_ylabel('Доля мошенничества')
    ax[0].set_title('Доля мошенничества по часам')
    
    # Распределение по дням недели
    dow_fraud = df_fe.groupby('day_of_week')['is_fraud'].mean()
    dow_fraud.plot(kind='bar', ax=ax[1], color='orange')
    ax[1].set_xlabel('День недели')
    ax[1].set_ylabel('Доля мошенничества')
    ax[1].set_title('Доля мошенничества по дням недели')
    ax[1].set_xticklabels(['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'], rotation=0)
    
    plt.tight_layout()
    plt.show()

## 8. Пропуски в данных

In [ ]:
# Процент пропусков
missing = df_fe.isnull().sum() / len(df_fe) * 100
missing = missing[missing > 0].sort_values(ascending=False)

if len(missing) > 0:
    print(f"Колонки с пропусками:")
    print(missing)
    
    plt.figure(figsize=(10, 6))
    missing.plot(kind='barh')
    plt.xlabel('Процент пропусков')
    plt.title('Пропуски в данных')
    plt.show()
else:
    print("✓ Пропусков нет")

## 9. Выводы

### Основные находки:

1. **Дисбаланс классов**: Мошеннические транзакции составляют малую долю от общего числа
2. **Распределение сумм**: Мошеннические транзакции могут иметь иное распределение сумм
3. **Временные паттерны**: Время суток и день недели могут быть важными признаками
4. **Поведенческие признаки**: Доступны богатые поведенческие данные

### Рекомендации:

- Использовать CatBoost/LightGBM (хорошо работают с дисбалансом)
- Подобрать оптимальный порог с учётом бизнес-требований
- Использовать F2-score для оценки (важнее не пропустить мошенничество)
- Применить SHAP для интерпретации решений